In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
import json
from datetime import datetime, timedelta

In [0]:
dbutils.widgets.dropdown("Load_type","DELTA" ,["FULL" ,"DELTA"] , "Load Type for Extraction and Loading")
dbutils.widgets.text("Flow","","Flow Name for Current Job")
dbutils.widgets.text("delta_load_time","10","Delta Load Time In days")
dbutils.widgets.text("Entity","orders","Entity Name")
dbutils.widgets.text("current_date","1900-01-01","Current Date")

In [0]:
load_type = dbutils.widgets.get("Load_type")
flow = dbutils.widgets.get("Flow")
delta_load_time = dbutils.widgets.get("delta_load_time")
entity = dbutils.widgets.get("Entity")
current_date = dbutils.widgets.get("current_date")

In [0]:
print("Running Load On: " + current_date)

In [0]:
if load_type.upper() not in ("FULL" , "DELTA"):
  raise Exception("Invalid Load Type")

try:
    delta_load_time = int(delta_load_time)
except:
  raise Exception("Invalid Delta Load Time")

In [0]:
create_path = "../sqls/creates/"
transformation_path = '../sqls/transformations/'

In [0]:
if load_type == "DELTA":
    delta_load_start = (datetime.strptime(current_date,"%Y-%m-%d") - timedelta(days=delta_load_time)).strftime("%Y-%m-%d")
else:
    delta_load_start = '1900-01-01'

print("Starting Loading data from:", delta_load_start)

### Staging Table Creation

In [0]:
with open(transformation_path + 'stg_' + entity + '.sql') as f:
  _content = f.read()

sql_content = _content.replace('{load_start}', delta_load_start) \
           .replace('{current_date}', current_date)


print("Read Content" + "\n\n\n" + sql_content)

In [0]:
sql_statements = sql_content.strip().split(';')
for statement in sql_statements:
    if statement != '' :
        print(statement)
        display(spark.sql(statement))

In [0]:
with open(transformation_path + entity + '_transformations.sql') as f:
  _content = f.read()

sql_content = _content.replace('{current_date}',current_date)

print("Read Content"+ "\n\n\n" + sql_content)

In [0]:
sql_statements = sql_content.strip().split(';')
for statement in sql_statements:
    if statement != '' :

        print(statement)

        display(spark.sql(statement))